In [7]:
import ROOT
from ROOT import gStyle
import numpy as np
import ctypes
import os

def biner(o_bin,m_bin,f_bin,f_step,l_step):
    a=np.arange(o_bin,m_bin,f_step)
    b=np.arange(m_bin,f_bin+l_step,l_step)
    return np.concatenate((a,b))

def normalization(hist_list):
    for hist in hist_list:
        for i in range(0,len(rebining)):
            value=hist.GetBinContent(i)
            error=hist.GetBinError(i)
            sf=hist.GetBinWidth(i)/5.0    
            hist.SetBinContent(i,value/sf)
            hist.SetBinError(i,error/sf)

In [8]:
histos={"inv_mass_topo":["yy",120,5,5,"Events/5 GeV","m(#mu_{1},#mu_{2})(GeV)",1.2,240],
        "h_inv_mass_topo_dphi_btag_iso_pt1_pt2_ptl":["yy",120,2,5,"Events/5 GeV","m(#mu_{1},#mu_{2})(GeV)",1.2,240,80,100],
        "Z_pt_cuts_ptl":["yy",180,5,10,"Events/5 GeV","Zp_{T}(GeV)",1.2,300],
        "lep1_pt_topo":["yy",180,5,10,"Events/5 GeV","p_{T}(#mu_{1})(GeV)",1.2,200],
        "lep2_pt_topo":["yy",180,5,10,"Events/5 GeV","p_{T}(#mu_{2})(GeV)",1.2,200],
        "lep1_pt_topo_dphi_btag_iso_pt2_mass_ptl":["yy",180,5,10,"Events/5 GeV","p_{T}(#mu_{1})(GeV)",1.2,200],
        "lep2_pt_topo_dphi_btag_iso_pt1_mass_ptl":["yy",180,5,10,"Events/5 GeV","p_{T}(#mu_{2})(GeV)",1.2,200],
        #"jet_n_topo_dphi_btag_iso_pt1_pt2_mass":["n",180,5,10,"Events","n-jets",1.2,10],
        #"jet_n_topo_dphi_btag_iso_pt1_pt2_mass_ptl":["n",180,5,10,"Events","n-jets",1.2,10],
}

In [10]:
######### CALCULATION OF RQCD #############

for k in histos:
    rebin=histos[k][0]
    histo=k

    MC_CR_OS=ROOT.TFile.Open("CR_OS/MC.root","READ")
    Data_CR_OS = ROOT.TFile.Open("CR_OS/Data.root","READ")
    MC_CR_SS=ROOT.TFile.Open("CR_SS/MC.root","READ")
    Data_CR_SS = ROOT.TFile.Open("CR_SS/Data.root","READ")

    a=k

    mc_cr_os=MC_CR_OS.Get(histo)
    data_cr_os=Data_CR_OS.Get(histo)
    mc_cr_ss=MC_CR_SS.Get(histo)
    data_cr_ss=Data_CR_SS.Get(histo)

    mc_cr_os.SetDirectory(0)
    data_cr_os.SetDirectory(0)
    mc_cr_ss.SetDirectory(0)
    data_cr_ss.SetDirectory(0)

    MC_CR_OS.Close()
    Data_CR_OS.Close()
    MC_CR_SS.Close()
    Data_CR_SS.Close()

    data_cr_os.Add(mc_cr_os,-1)
    data_cr_ss.Add(mc_cr_ss,-1)

    RQCD=data_cr_os.Integral()/data_cr_ss.Integral()
    print(k)
    print(RQCD)

    # HISTOGRAM FOR SR_OS

    MC_SR_OS=ROOT.TFile.Open("SR_OS/MC.root","READ")
    Data_SR_OS = ROOT.TFile.Open("SR_OS/Data.root","READ")
    MC_SR_SS=ROOT.TFile.Open("SR_SS/MC.root","READ")
    Data_SR_SS = ROOT.TFile.Open("SR_SS/Data.root","READ")

    mc_sr_os=MC_SR_OS.Get(histo)
    data_sr_os=Data_SR_OS.Get(histo)
    mc_sr_ss=MC_SR_SS.Get(histo)
    data_sr_ss=Data_SR_SS.Get(histo)

    mc_sr_os.SetDirectory(0)
    data_sr_os.SetDirectory(0)
    mc_sr_ss.SetDirectory(0)
    data_sr_ss.SetDirectory(0)

    MC_SR_OS.Close()
    Data_SR_OS.Close()
    MC_SR_SS.Close()
    Data_SR_SS.Close()

    ob=data_sr_ss.GetNbinsX()
    s=data_sr_ss.GetXaxis().GetBinLowEdge(1)
    e=data_sr_ss.GetXaxis().GetBinUpEdge(data_sr_ss.GetNbinsX())
    n=5
    if rebin=="yy" or rebin=="yn":
        rebining=biner(s,histos[k][1],e,histos[k][2],histos[k][3])
        nb=len(rebining)-1
        mc_sr_ss=mc_sr_ss.Rebin(nb,"mc_sr_ss",rebining)
        data_sr_ss=data_sr_ss.Rebin(nb,"data_sr_ss",rebining)


    MJ_BG=data_sr_ss.Clone()
    MJ_BG.Add(mc_sr_ss,-1)
    for kk in range(1,MJ_BG.GetNbinsX()+1):
        if MJ_BG.GetBinContent(kk)<0.0:
            MJ_BG.SetBinContent(kk,0.0)
            
    print("SR_SS Yield",MJ_BG.Integral(),"ERROR= ",RQCD*(MJ_BG.Integral())**0.5)
    MJ_BG.Scale(RQCD)

    # SAVING MJ BG HISTOGRAM

    ##############################
    files=os.listdir('SR_OS')
    print(files)
    if 'MJ.root' in files:
        os.system('rm SR_OS/MJ.root')
    
    file=ROOT.TFile.Open("SR_OS/MJ.root","update")
    MJ_BG.Write(a)
    file.Close()
    ##############################

    #PLOTS

    ##### LOADING INFORMATION ABOUT EACH SAMPLE #####
    
    Data = ROOT.TFile.Open("SR_OS/Data.root","READ")
    Signal=ROOT.TFile.Open("SR_OS/Zee.root","READ")
    VV=ROOT.TFile.Open("SR_OS/VV.root","READ")
    Wjets=ROOT.TFile.Open("SR_OS/Wjets.root","READ")
    Zjets=ROOT.TFile.Open("SR_OS/Ztautau_Sherpa.root","READ")
    Ttbar=ROOT.TFile.Open("SR_OS/ttbar.root","READ")
    SingleTop=ROOT.TFile.Open("SR_OS/singletop.root","READ")
    MJ=ROOT.TFile.Open("SR_OS/MJ.root","READ")
    Error=ROOT.TFile.Open("SR_OS/MC.root","READ")

    data=Data.Get(a)
    signal=Signal.Get(a)
    vv=VV.Get(a)
    w=Wjets.Get(a)
    z=Zjets.Get(a)
    tt=Ttbar.Get(a)
    t=SingleTop.Get(a)
    mj=MJ.Get(a)
    error=Error.Get(a)

    signal.SetDirectory(0)
    data.SetDirectory(0)
    vv.SetDirectory(0)
    w.SetDirectory(0)
    z.SetDirectory(0)
    tt.SetDirectory(0)
    t.SetDirectory(0)
    mj.SetDirectory(0)
    error.SetDirectory(0)

    Signal.Close()
    Data.Close()
    VV.Close()
    Wjets.Close()
    Zjets.Close()
    Ttbar.Close()
    SingleTop.Close()
    MJ.Close()
    Error.Close()
    
    ####### DECLARING STACK HISTOGRAM ######
    
    hs = ROOT.THStack("hs","")
    
    ###### REBINING HISTOGRAMS #######
    
    if rebin=="yy" or rebin=="yn":
        signal=signal.Rebin(nb,"signal",rebining)
        data=data.Rebin(nb,"data",rebining)
        vv=vv.Rebin(nb,"vv",rebining)
        tt=tt.Rebin(nb,"tt",rebining)
        t=t.Rebin(nb,"t",rebining)
        z=z.Rebin(nb,"z",rebining)
        w=w.Rebin(nb,"w",rebining)
        error=error.Rebin(nb,"error",rebining)

    ###### CALCULATING THE RATIO MC/DATA #######    
        
    ratio=w.Clone()
    lista = ROOT.TList()
    lista.Add(mj)
    lista.Add(z)
    lista.Add(vv)
    lista.Add(tt)
    lista.Add(t)
    lista.Add(signal)
    ratio.Merge(lista)
    ratio.Divide(data)
    ratio_error=ratio.Clone()
    
    ###### NORMALIZING HISTOGRAM USING VARIABLE BINING #######

    if rebin=="yy":
        hist_list=[signal,data,vv,tt,t,z,w,mj,error]
        normalization(hist_list)
       
    ###### ADDING COLOURS TO THE HISTOGRAMS #######
        
    signal.SetFillColor(ROOT.kAzure+8)
    w.SetFillColor(ROOT.kGreen)
    w.SetLineColor(ROOT.kGreen+1)
    z.SetFillColor(ROOT.kRed)
    z.SetLineColor(ROOT.kRed+1)
    vv.SetFillColor(ROOT.kBlue)
    vv.SetLineColor(ROOT.kBlue+1)
    tt.SetFillColor(ROOT.kYellow)
    tt.SetLineColor(ROOT.kYellow+1)
    t.SetFillColor(ROOT.kCyan)
    t.SetLineColor(ROOT.kCyan+1)
    mj.SetLineColor(ROOT.kMagenta+1)
    mj.SetFillColor(ROOT.kMagenta)
    data.SetLineColor(ROOT.kBlack)
    error.SetFillColorAlpha(ROOT.kBlue,0.5)
    ratio_error.SetFillColorAlpha(ROOT.kBlue,0.5)
    
    ###### CREATING STACK HISTOGRAM #######
    
    hs.Add(vv)
    hs.Add(w)
    hs.Add(z)
    hs.Add(tt)
    hs.Add(t)
    hs.Add(mj)
    hs.Add(signal)
    
    error.Add(mj,1)

    ##### DRAWING THE STATISTICS BOX #####
    
    gStyle.SetOptStat(1111111)

    ##### DRAWING CANVAS #######
    
    canvas = ROOT.TCanvas("canvas")
    canvas.cd()
    
    ##### DRAWING TOP PAD, SETTING MARGINS #######

    pad1 = ROOT . TPad (" pad1 "," pad1 " ,0 ,0.35 ,1 ,1)
    pad1.SetTopMargin(0.03)
    pad1.SetRightMargin(0.03)
    pad1.SetLeftMargin(0.08)
    pad1.SetBottomMargin(0.0)
    pad1.Draw ()
    pad1.cd ()

    ###### SETTING STATS BOX POSITION ######
    
    gStyle.SetStatY(0.93);                
    gStyle.SetStatX(0.95);
    gStyle.SetStatW(0.1);                
    gStyle.SetStatH(0.1);
    
    #### OPTION FOR LOG PLOT #####
    
    if "inv_mass" in k:
        pad1.SetLogy(1)

    #### DRAWING HISTOGRAMS #####    
        
    data.SetMarkerStyle(ROOT.kFullCircle)
    data.SetMarkerSize(0.7)
    data.Draw("pe")
    error.Draw("E2 same")
    hs.Draw("HIST same")
    error.Draw("E2 same")
    data.Draw("pe same")
    
    ###### DRAWING CUT LINES ######
    
    if len(histos[k])>8:
        cut1 = ROOT . TLine (histos[k][8] ,0 ,histos[k][8],histos[k][6]*data.GetBinContent(data.GetMaximumBin()))
        cut1 . SetLineColor ( ROOT . kRed+1 )
        cut1 . SetLineWidth (2)
        cut1.SetLineStyle(2)
        
        cut2 = ROOT . TLine (histos[k][9] ,0 ,histos[k][9],histos[k][6]*data.GetBinContent(data.GetMaximumBin()))
        cut2 . SetLineColor ( ROOT . kRed+1 )
        cut2 . SetLineWidth (2)
        cut2.SetLineStyle(2)
        
        cut1.Draw("same")
        cut2.Draw("same")
    
    data.GetXaxis().Draw()
    pad1.RedrawAxis()

    ###### ADDING A LABEL ACCORDING TO THE CHANNEL ######
    
    data.SetTitle("")
    l=ROOT.TLatex()
    l.SetNDC ()
    l.DrawLatex(0.6,0.88,"Z#rightarrow#tau#tau#rightarrow#tau#mu")
    
    ##### ADJUSTING THE RANGE AND THE TITLE OF THE AXIS #####
    
    if len(histos[k])>7:
        e=histos[k][7]

    pad1.SetTicks(1,1)
    pad1.Update()
    data . GetYaxis (). SetRangeUser (1 ,histos[k][6]*data.GetBinContent(data.GetMaximumBin()))
    data . GetXaxis (). SetRangeUser (s ,e)
    data . GetYaxis (). SetTitle (histos[k][4])
    data . GetXaxis (). SetTitle (histos[k][5])
    
    ###### SETTING THE LEGENDS ON THE PLOT #####

    hs . GetXaxis (). SetRangeUser (s ,e)
    legend = ROOT . TLegend (0.7 ,0.73 ,0.8 ,0.93)
    legend . AddEntry ( vv ,"Diboson")
    legend . AddEntry ( data ," Data ")
    legend . AddEntry ( signal ," Signal ")
    legend . AddEntry ( w ," W+jets ")
    legend . AddEntry ( z ," Z+jet ")
    legend . AddEntry ( t ," Single top ")
    legend . AddEntry ( tt ," ttbar ")
    legend . AddEntry ( mj,"MJ")
    legend . AddEntry ( error,"MC Stat. Error")
    gStyle.SetLegendBorderSize(0)
    legend . SetLineWidth (2)
    legend . Draw ()
    hs.SetTitle(a)

    ###### DRAWING PAD2 OF THE CANVAS, DRAWING THE RATIO MC/DATA ######
    
    canvas.cd()
    pad2 = ROOT . TPad (" pad2 "," pad2 " ,0 ,0.00 ,1 ,0.35)
    pad2.SetRightMargin(0.03)
    pad2.SetLeftMargin(0.08)
    pad2.SetTopMargin(0)
    pad2.SetBottomMargin(0.2)
    pad2.Draw ()
    pad2.cd ()
    ratio.SetMarkerStyle(ROOT.kFullCircle)
    ratio.SetMarkerSize(0.7)
    ratio.Draw ("p HIST")
    ratio_error.Draw ("E2 same")
    ratio.Draw ("p hist same")
    ratio.SetStats(0)
    ratio.SetStats(0)
    ratio.SetTitle("")
    ratio . GetYaxis (). SetRangeUser (0 ,2.0)
    ratio . GetXaxis (). SetRangeUser (s ,e)
    ratio . GetYaxis (). SetTitle ("MC/DATA")
    ratio . GetYaxis (). SetTitleSize (0.10)
    ratio . GetYaxis (). SetTitleOffset (0.25)
    ratio . GetXaxis (). SetTitle (histos[k][5])
    ratio . GetXaxis (). SetTitleSize (0.09)
    ratio.GetXaxis().SetLabelSize(0.10)
    ratio.GetYaxis().SetLabelSize(0.08)
    
    ###### SETTING ALL THE HORIZONTAL DASHED LINES #######
    
    line = ROOT . TLine (s ,1 ,e,1)
    line . SetLineColor ( ROOT . kBlack )
    line . SetLineWidth (2)
    line2 = ROOT . TLine (s ,1.2 ,e,1.2)
    line2 . SetLineColor ( ROOT . kBlack )
    line2 . SetLineWidth (1)
    line2 . SetLineStyle(2)
    line3 = ROOT . TLine (s ,0.8 ,e,0.8)
    line3 . SetLineColor ( ROOT . kBlack )
    line3 . SetLineWidth (1)
    line3 . SetLineStyle(2)
    line4 = ROOT . TLine (s ,1.4 ,e,1.4)
    line4 . SetLineColor ( ROOT . kBlack )
    line4 . SetLineWidth (1)
    line4 . SetLineStyle(2)
    line5 = ROOT . TLine (s ,0.6 ,e,0.6)
    line5 . SetLineColor ( ROOT . kBlack )
    line5 . SetLineWidth (1)
    line5 . SetLineStyle(2)
    line6 = ROOT . TLine (s ,1.6 ,e,1.6)
    line6 . SetLineColor ( ROOT . kBlack )
    line6 . SetLineWidth (1)
    line6 . SetLineStyle(2)
    line7 = ROOT . TLine (s ,0.4 ,e,0.4)
    line7 . SetLineColor ( ROOT . kBlack )
    line7 . SetLineWidth (1)
    line7 . SetLineStyle(2)
    line8 = ROOT . TLine (s ,1.8 ,e,1.8)
    line8 . SetLineColor ( ROOT . kBlack )
    line8 . SetLineWidth (1)
    line8 . SetLineStyle(2)
    line9 = ROOT . TLine (s ,0.2 ,e,0.2)
    line9 . SetLineColor ( ROOT . kBlack )
    line9 . SetLineWidth (1)
    line9 . SetLineStyle(2)
    line . Draw (" same ")
    line2 . Draw (" same ")
    line3 . Draw (" same ")
    line4 . Draw (" same ")
    line5 . Draw (" same ")
    line6 . Draw (" same ")
    line7 . Draw (" same ")
    line8 . Draw (" same ")
    line9 . Draw (" same ")
    line . Draw (" same ")
    
    ##### DRAWING THE CUT LINES IN CASE IT APPLIES #######
    
    if len(histos[k])>8:
        cut1.Draw("same")
        cut2.Draw("same")
    pad2.RedrawAxis()
    
    ###### SAVING THE HISTOGRAMS AS PDF FILES ########
    
    canvas.Print(a+".pdf")
    
    ####### PRINTING THE YIELDS FOR EVERY SAMPLE WITH THE STAT ERRORS #######
    
    if k=="ratio_lpt_tpt_cuts_tpt":
        dic={"data":data,"signal":signal,"vv":vv,"w":w,"z":z,"tt":tt,"t":t,"mj":mj}
        for i in dic:
            error=ctypes.c_double()
            a=dic[i].IntegralAndError(0,1000,error)
            print(i)
            print(str(a)+" +E "+str(error))

inv_mass_topo
559.5101274409676
SR_SS Yield 4943.68550491333 ERROR=  39339.910672607
['lep1_phi_topo.pdf', 'h_inv_mass_topo_dphi_btag_iso_pt1_pt2_ptl.pdf', 'Wjets.root', 'Zmumu.root', 'sum_pt_pt_cuts_ptl_ZpTb.pdf', 'singletop.root', 'Zee.root', 'ratio_lpt_tpt_cuts_ptl.pdf', 'lep2_phi_topo.pdf', 'lep2_pt_topo.pdf', 'sumlep_pt_topo.pdf', 'lep1_pt_topo.pdf', 'delta_phi_topo.pdf', 'MJ.root', 'inv_mass_topo.pdf', 'Ztautau_PoPy.root', 'lep2_pt_topo_dphi_btag_iso_pt1_mass_ptl.pdf', 'lep2_phi_cuts.pdf', 'ttbar.root', 'delta_phi_cuts_butphi.pdf', 'Ztautau_Sherpa.root', 'lep1_phi_cuts.pdf', 'jet_n_topo_dphi_btag_iso_pt1_pt2_mass.pdf', 'sum_pt_pt_cuts_ZpTc.pdf', 'Z_pt_topo.pdf', 'ratio_lpt_tpt_cuts.pdf', 'Data.root', 'sum_pt_pt_cuts_ZpTa.pdf', 'sum_pt_pt_cuts_ZpTb.pdf', 'BG.root', 'Z_pt_cuts.pdf', 'lep1_pt_topo_dphi_btag_iso_pt2_mass_ptl.pdf', 'sum_pt_pt_cuts_ptl_ZpTc.pdf', 'ratio_ptjet_zpt_cuts_ptl.pdf', 'VV.root', 'sumlep_pt_topo_dphi_btag_iso_pt1_pt2_mass_ptl.pdf', 'ratio_ptjet_zpt_cuts.pdf', 

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file inv_mass_topo.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file h_inv_mass_topo_dphi_btag_iso_pt1_pt2_ptl.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file Z_pt_cuts_ptl.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file lep1_pt_topo.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file lep2_pt_topo.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file lep1_pt_topo_dphi_btag_iso_pt2_mass_ptl.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <T

In [ ]:
canvas = ROOT.TCanvas("canvas")
canvas.cd()



In [4]:
import os
import shutil
if 'OnlySignal' in os.listdir():
    shutil.rmtree('OnlySignal')
os.mkdir('OnlySignal')
for k in histos:
    rebin=histos[k][0]
    histo=k

    MC_CR_OS=ROOT.TFile.Open("CR_OS/MC.root","READ")
    Data_CR_OS = ROOT.TFile.Open("CR_OS/Data.root","READ")
    MC_CR_SS=ROOT.TFile.Open("CR_SS/MC.root","READ")
    Data_CR_SS = ROOT.TFile.Open("CR_SS/Data.root","READ")

    a=k

    mc_cr_os=MC_CR_OS.Get(histo)
    data_cr_os=Data_CR_OS.Get(histo)
    mc_cr_ss=MC_CR_SS.Get(histo)
    data_cr_ss=Data_CR_SS.Get(histo)

    mc_cr_os.SetDirectory(0)
    data_cr_os.SetDirectory(0)
    mc_cr_ss.SetDirectory(0)
    data_cr_ss.SetDirectory(0)

    MC_CR_OS.Close()
    Data_CR_OS.Close()
    MC_CR_SS.Close()
    Data_CR_SS.Close()

    data_cr_os.Add(mc_cr_os,-1)
    data_cr_ss.Add(mc_cr_ss,-1)

    RQCD=data_cr_os.Integral()/data_cr_ss.Integral()

    # HISTOGRAM FOR SR_OS

    MC_SR_OS=ROOT.TFile.Open("SR_OS/MC.root","READ")
    Data_SR_OS = ROOT.TFile.Open("SR_OS/Data.root","READ")
    MC_SR_SS=ROOT.TFile.Open("SR_SS/MC.root","READ")
    Data_SR_SS = ROOT.TFile.Open("SR_SS/Data.root","READ")

    mc_sr_os=MC_SR_OS.Get(histo)
    data_sr_os=Data_SR_OS.Get(histo)
    mc_sr_ss=MC_SR_SS.Get(histo)
    data_sr_ss=Data_SR_SS.Get(histo)

    mc_sr_os.SetDirectory(0)
    data_sr_os.SetDirectory(0)
    mc_sr_ss.SetDirectory(0)
    data_sr_ss.SetDirectory(0)

    MC_SR_OS.Close()
    Data_SR_OS.Close()
    MC_SR_SS.Close()
    Data_SR_SS.Close()

    ob=data_sr_ss.GetNbinsX()
    s=data_sr_ss.GetXaxis().GetBinLowEdge(1)
    e=data_sr_ss.GetXaxis().GetBinUpEdge(data_sr_ss.GetNbinsX())
    n=5
    if rebin=="yy" or rebin=="yn":
        rebining=biner(s,histos[k][1],e,histos[k][2],histos[k][3])
        nb=len(rebining)-1
        mc_sr_ss=mc_sr_ss.Rebin(nb,"mc_sr_ss",rebining)
        data_sr_ss=data_sr_ss.Rebin(nb,"data_sr_ss",rebining)


    MJ_BG=data_sr_ss.Clone()
    MJ_BG.Add(mc_sr_ss,-1)
    MJ_BG.Scale(RQCD)

    # SAVING MJ BG HISTOGRAM

    ##############################
    file=ROOT.TFile.Open("SR_OS/MJ.root","update")
    MJ_BG.Write(a)
    file.Close()
    ##############################

    #PLOTS

    
    ##### LOADING INFORMATION ABOUT EACH SAMPLE #####
    
    Data = ROOT.TFile.Open("SR_OS/Data.root","READ")
    Signal=ROOT.TFile.Open("SR_OS/Signal.root","READ")
    VV=ROOT.TFile.Open("SR_OS/VV.root","READ")
    Wjets=ROOT.TFile.Open("SR_OS/Wjets.root","READ")
    Zjets=ROOT.TFile.Open("SR_OS/Zjet.root","READ")
    Ttbar=ROOT.TFile.Open("SR_OS/ttbar.root","READ")
    SingleTop=ROOT.TFile.Open("SR_OS/singletop.root","READ")
    MJ=ROOT.TFile.Open("SR_OS/MJ.root","READ")
    Error=ROOT.TFile.Open("SR_OS/Signal.root","READ")

    data=Data.Get(a)
    signal=Signal.Get(a)
    vv=VV.Get(a)
    w=Wjets.Get(a)
    z=Zjets.Get(a)
    tt=Ttbar.Get(a)
    t=SingleTop.Get(a)
    mj=MJ.Get(a)
    error=Error.Get(a)

    signal.SetDirectory(0)
    data.SetDirectory(0)
    vv.SetDirectory(0)
    w.SetDirectory(0)
    z.SetDirectory(0)
    tt.SetDirectory(0)
    t.SetDirectory(0)
    mj.SetDirectory(0)
    error.SetDirectory(0)

    Signal.Close()
    Data.Close()
    VV.Close()
    Wjets.Close()
    Zjets.Close()
    Ttbar.Close()
    SingleTop.Close()
    MJ.Close()
    Error.Close()
    
    ##### REBINING HISTOGRAMS #####

    if rebin=="yy" or rebin=="yn":
        signal=signal.Rebin(nb,"signal",rebining)
        data=data.Rebin(nb,"data",rebining)
        vv=vv.Rebin(nb,"vv",rebining)
        tt=tt.Rebin(nb,"tt",rebining)
        t=t.Rebin(nb,"t",rebining)
        z=z.Rebin(nb,"z",rebining)
        w=w.Rebin(nb,"w",rebining)
        error=error.Rebin(nb,"error",rebining)
    
    ##### IF HISTOGRAMS HAVE VARIABLE BINNING, NORMALIZE IT #####
    
    if rebin=="yy":
        hist_list=[signal,data,vv,tt,t,z,w,mj,error]
        normalization(hist_list)
        
    
    ##### SUBSTRACTING EW BG FROM DATA #####
    
    data_minus_BG=data.Clone()
    data_minus_BG.Add(w,-1)
    data_minus_BG.Add(z,-1)
    data_minus_BG.Add(vv,-1)
    data_minus_BG.Add(tt,-1)
    data_minus_BG.Add(t,-1)
    data_minus_BG.Add(mj,-1)
    
    ##### DEFINING RATIO SIGNAL VS DATA-BG #####
    
    ratio=signal.Clone()
    ratio.Divide(data_minus_BG)
    ratio_error=ratio.Clone()
    
    ##### COLOURING THE HISTOGRAMS ##### 
    
    signal.SetFillColor(ROOT.kAzure+8)
    data_minus_BG.SetLineColor(ROOT.kBlack)
    error.SetFillColorAlpha(ROOT.kBlue,0.5)
    ratio_error.SetFillColorAlpha(ROOT.kBlue,0.5)
    
    ##### DRAWING THE STATISTICS BOX #####
    gStyle.SetOptStat(1111111)

    
    ##### DRAWING CANVAS #######
    canvas = ROOT.TCanvas("canvas")
    canvas.cd()
    
    ##### DRAWING TOP PAD, SETTING MARGINS #######

    pad1 = ROOT . TPad (" pad1 "," pad1 " ,0 ,0.35 ,1 ,1)
    pad1.SetTopMargin(0.03)
    pad1.SetRightMargin(0.03)
    pad1.SetLeftMargin(0.08)
    pad1.SetBottomMargin(0.0)
    pad1.Draw ()
    pad1.cd ()
    
    ##### SETTING STATS BOX POSITION #######

    gStyle.SetStatY(0.93);                
    gStyle.SetStatX(0.95);
    gStyle.SetStatW(0.1);                
    gStyle.SetStatH(0.1);
    
    #### OPTION FOR LOG PLOT #####
    
    #if "rnn_score" in k:
        #pad1.SetLogy(1)

    #### DRAWING HISTOGRAMS #####    
    
    data_minus_BG.SetMarkerStyle(ROOT.kFullCircle)
    data_minus_BG.SetMarkerSize(0.7)
    data_minus_BG.Draw("pe")
    signal.Draw("HIST same")
    error.Draw("E2 same")
    data_minus_BG.Draw("pe same")
    
    
    #### DRAWING CUT LINES #####  
    
    if len(histos[k])>8:
        cut1 = ROOT . TLine (histos[k][8] ,0 ,histos[k][8],histos[k][6]*data.GetBinContent(data.GetMaximumBin()))
        cut1 . SetLineColor ( ROOT . kRed+1 )
        cut1 . SetLineWidth (2)
        cut1.SetLineStyle(2)
        
        cut2 = ROOT . TLine (histos[k][9] ,0 ,histos[k][9],histos[k][6]*data.GetBinContent(data.GetMaximumBin()))
        cut2 . SetLineColor ( ROOT . kRed+1 )
        cut2 . SetLineWidth (2)
        cut2.SetLineStyle(2)
        
        cut1.Draw("same")
        cut2.Draw("same")
    
    data_minus_BG.GetXaxis().Draw()
    pad1.RedrawAxis()

    ###### ADDING A LABEL ACCORDING TO THE CHANNEL ######
    
    data_minus_BG.SetTitle("")
    l=ROOT.TLatex()
    l.SetNDC ()
    l.DrawLatex(0.6,0.88,"Z#rightarrow#tau#tau#rightarrow#tau#mu")
    
    ##### ADJUSTING THE RANGE AND THE TITLE OF THE AXIS #####
    
    if len(histos[k])>7:
        e=histos[k][7]

    pad1.SetTicks(1,1)
    pad1.Update()
    data_minus_BG.GetYaxis (). SetRangeUser (1 ,histos[k][6]*data_minus_BG.GetBinContent(data_minus_BG.GetMaximumBin()))
    data_minus_BG.GetXaxis (). SetRangeUser (s ,e)
    data_minus_BG.GetYaxis (). SetTitle (histos[k][4])
    data_minus_BG.GetXaxis (). SetTitle (histos[k][5])

    ###### SETTING THE LEGENDS ON THE PLOT #####
    
    legend = ROOT . TLegend (0.7 ,0.73 ,0.8 ,0.93)
    legend . AddEntry ( data_minus_BG ," Data ")
    legend . AddEntry ( signal ," Signal ")
    legend . AddEntry ( error,"MC Stat. Error")
    gStyle.SetLegendBorderSize(0)
    legend . SetLineWidth (2)
    legend . Draw ()
    
    ###### DRAWING PAD2 OF THE CANVAS, DRAWING THE RATIO MC/DATA ######

    canvas.cd()
    pad2 = ROOT . TPad (" pad2 "," pad2 " ,0 ,0.00 ,1 ,0.35)
    pad2.SetRightMargin(0.03)
    pad2.SetLeftMargin(0.08)
    pad2.SetTopMargin(0)
    pad2.SetBottomMargin(0.2)
    pad2.Draw ()
    pad2.cd ()
    ratio.SetMarkerStyle(ROOT.kFullCircle)
    ratio.SetMarkerSize(0.7)
    ratio.Draw ("p HIST")
    ratio_error.Draw ("E2 same")
    ratio.Draw ("p hist same")
    ratio.SetStats(0)
    ratio.SetStats(0)
    ratio.SetTitle("")
    ratio . GetYaxis (). SetRangeUser (0 ,2.0)
    ratio . GetXaxis (). SetRangeUser (s ,e)
    ratio . GetYaxis (). SetTitle ("MC/DATA")
    ratio . GetYaxis (). SetTitleSize (0.10)
    ratio . GetYaxis (). SetTitleOffset (0.25)
    ratio . GetXaxis (). SetTitle (histos[k][5])
    ratio . GetXaxis (). SetTitleSize (0.09)
    ratio.GetXaxis().SetLabelSize(0.10)
    ratio.GetYaxis().SetLabelSize(0.08)
    
    ###### SETTING ALL THE HORIZONTAL DASHED LINES #######
    
    line = ROOT . TLine (s ,1 ,e,1)
    line . SetLineColor ( ROOT . kBlack )
    line . SetLineWidth (2)
    line2 = ROOT . TLine (s ,1.2 ,e,1.2)
    line2 . SetLineColor ( ROOT . kBlack )
    line2 . SetLineWidth (1)
    line2 . SetLineStyle(2)
    line3 = ROOT . TLine (s ,0.8 ,e,0.8)
    line3 . SetLineColor ( ROOT . kBlack )
    line3 . SetLineWidth (1)
    line3 . SetLineStyle(2)
    line4 = ROOT . TLine (s ,1.4 ,e,1.4)
    line4 . SetLineColor ( ROOT . kBlack )
    line4 . SetLineWidth (1)
    line4 . SetLineStyle(2)
    line5 = ROOT . TLine (s ,0.6 ,e,0.6)
    line5 . SetLineColor ( ROOT . kBlack )
    line5 . SetLineWidth (1)
    line5 . SetLineStyle(2)
    line6 = ROOT . TLine (s ,1.6 ,e,1.6)
    line6 . SetLineColor ( ROOT . kBlack )
    line6 . SetLineWidth (1)
    line6 . SetLineStyle(2)
    line7 = ROOT . TLine (s ,0.4 ,e,0.4)
    line7 . SetLineColor ( ROOT . kBlack )
    line7 . SetLineWidth (1)
    line7 . SetLineStyle(2)
    line8 = ROOT . TLine (s ,1.8 ,e,1.8)
    line8 . SetLineColor ( ROOT . kBlack )
    line8 . SetLineWidth (1)
    line8 . SetLineStyle(2)
    line9 = ROOT . TLine (s ,0.2 ,e,0.2)
    line9 . SetLineColor ( ROOT . kBlack )
    line9 . SetLineWidth (1)
    line9 . SetLineStyle(2)
    line . Draw (" same ")
    line2 . Draw (" same ")
    line3 . Draw (" same ")
    line4 . Draw (" same ")
    line5 . Draw (" same ")
    line6 . Draw (" same ")
    line7 . Draw (" same ")
    line8 . Draw (" same ")
    line9 . Draw (" same ")
    line . Draw (" same ")
    
    ##### DRAWING THE CUT LINES IN CASE IT APPLIES #######
    
    if len(histos[k])>8:
        cut1.Draw("same")
        cut2.Draw("same")
    pad2.RedrawAxis()
    
    ###### SAVING THE HISTOGRAMS AS PDF FILES ########
    
    os.chdir('OnlySignal/')
    canvas.Print(a+".pdf")
    os.chdir('../')

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file tau_pt_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_inside.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file tau_pt_topo_dphi_btag_iso_rnn_ptmu_omega_mreco_outside.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file lep_pt0_topo_dphi_btag_iso_rnn_ptmu_omega_mreco.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file MET_topo_dphi_btag_iso_rnn_ptmu_omega_mreco.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file Z_pt_cuts_inside.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file Z_pt_cuts_outside.pdf h